In [1]:
# Not needed if tinc-python is installed
import sys
sys.path.append('../external/tinc/tinc-python/tinc-python')

In [2]:
from tinc_client import *
tclient = TincClient()

Attempt connection. localhost:34450


In [4]:
tclient.synchronize()

ParameterSpace already registered: 'tinc::IdObject@2405450683896'
ParameterSpace already registered: 'tinc::IdObject@2405450683896'
ParameterSpace already registered: 'tinc::IdObject@2405450683896'


In [5]:
tclient.print()

Print
TINC Server
CONNECTED
 ** Parameter ShellSiteTypes group:  (<class 'int'>)
    Default: 0
    Min: 0
    Max: 127
 ** Parameter markerColor group:  (<class 'list'>)
    Default: [1.0, 1.0, 1.0, 1.0]
    Min: [0, 0, 0, 0]
    Max: [1, 1, 1, 1]
 ** Parameter markerScale group:  (<class 'float'>)
    Default: 0.0
    Min: 0.0010000000474974513
    Max: 20.0
 ** Parameter dataset group:  (<class 'str'>)
    Default: 
 ** ParameterSpace tinc::IdObject@2405450683896
 ** DiskBuffer: 'graph' type 0
      path: C:\Users\Andres\source\repos\casm_viewer\bin\cachedGraph\ basename: currentGraph.png
*** Computation Chain: PrepareDataset
*** Data Script Processor: ParameterSpaceProcessor
Output files: ['parameter_space.nc']
*** Data Script Processor: ShellSiteFileAnalyzer
*** Data Script Processor: TransfmatExtractor
Output files: ['transfmat']
*** Data Script Processor: TemplateGenerator
Output files: ['cached_output\\template.nc']
*** Computation Chain: SampleComputation
*** Computation Chain

In [7]:
tclient.parameter_spaces[0].get_current_path()

'90\\conditions.0'

In [8]:
tclient.parameter_spaces[0].get_root_path()

'C:\\Users\\Andres\\source\\repos\\vdv_data\\new\\kinetic_mc\\'

The 'graph' ```imageBuffer``` determines what graph is shown.

In [9]:
imageBuffer = tclient.get_disk_buffer('graph')

In [10]:
imageBuffer._path

'C:\\Users\\Andres\\source\\repos\\casm_viewer\\bin\\cachedGraph\\'

In [13]:
import random
import matplotlib.pyplot as plt
import threading

def update_graph(parameter_value):
    print("Parameter value " + str(parameter_value))
    data = [random.random() * parameter_value for i in range(10)]
    with threading.Lock():
        fname = imageBuffer.get_filename_for_writing()
        f = plt.figure()
        plt.title(f" Time: {parameter_value}")
        plt.plot(data)
        #print("Update " + fname)
        plt.savefig(fname)
        # The following to avoid extra printout in jupyter cell
        plt.close()
        f.clf()
        del f
        imageBuffer.done_writing_file(fname)

In [14]:
update_graph(40)

Parameter value 40


In [15]:
shellsites = tclient.get_parameter("ShellSiteTypes")

In [16]:
tclient.command_parameter_choice_elements(shellsites)

['cleared_path0.nc', 'double_cleared_path0.nc', 'forced_path0.nc', 'free_range_path0.nc', 'oct_doub_va0.nc', 'oct_no_va0.nc', 'oct_sing_va0.nc', 'oct_trip_va0.nc', 'queued_path0.nc', 'semiforced_path0.nc', 'tet_no_va0.nc', 'tet_sing_va0.nc', 'trapped_path0.nc']


In [17]:
shellsites.get_current_elements()

[]

In [18]:
shellsites.register_callback(update_graph)

In [24]:
shellsites.value

0

In [25]:
ms = tclient.get_parameter("markerScale")

In [26]:
ms.value

20.0

In [27]:
ms.value = 20

In [28]:
mc = tclient.get_parameter("markerColor")

In [29]:
mc.value

[1.0, 1.0, 1.0, 1.0]

In [30]:
mc.value = [0.5, 1, 0.5, 1]

In [31]:
import matplotlib

In [32]:
matplotlib.colors.to_rgb('blue')

(0.0, 0.0, 1.0)

In [33]:
colors = ['blue', 'red', 'green', 'white', 'darkBlue', 'cyan', 'magenta', 'lightBlue','blue', 'red', 'green', 'cyan', 'magenta', 'lightBlue']

In [34]:
[matplotlib.colors.to_rgb(c) for c in colors]

[(0.0, 0.0, 1.0),
 (1.0, 0.0, 0.0),
 (0.0, 0.5019607843137255, 0.0),
 (1.0, 1.0, 1.0),
 (0.0, 0.0, 0.5450980392156862),
 (0.0, 1.0, 1.0),
 (1.0, 0.0, 1.0),
 (0.6784313725490196, 0.8470588235294118, 0.9019607843137255),
 (0.0, 0.0, 1.0),
 (1.0, 0.0, 0.0),
 (0.0, 0.5019607843137255, 0.0),
 (0.0, 1.0, 1.0),
 (1.0, 0.0, 1.0),
 (0.6784313725490196, 0.8470588235294118, 0.9019607843137255)]

In [35]:
bin(shellsites.value)

'0b0'

In [36]:
def set_color(shellSite):
    b = int(shellSite)
    for c in colors:
        if b & 1 == 1:
            rgb = list(matplotlib.colors.to_rgb(c))
            rgb.append(1)
            mc.value = rgb
            break
        b = b >> 1
    

In [37]:
set_color(1024)

In [38]:
shellsites.register_callback(set_color)

Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0
Parameter value 0


## Datapools

In [39]:
[dp.id for dp in tclient.datapools]

['resultsData', 'trajectories', 'neighborhood']

In [9]:
dp = tclient.get_datapool("resultsData")

In [10]:
dp.slice_cache_dir

'C:\\Users\\Andres\\source\\repos\\vdv_data\\new\\kinetic_mc\\slices\\'

Connection closed.Connection closed.
Attempt connection. localhost:34450

Attempt connection. localhost:34450


In [10]:
dp.get_slice("<comp(a)>", "chempotA")

masked_array(data=[1.21477162e-04, 1.21477162e-04, 1.21477162e-04,
                   4.85908648e-04, 4.85908648e-04, 6.07385824e-04,
                   4.85908648e-04, 2.42954324e-04, 6.07385824e-04,
                   8.50340119e-04, 1.33624882e-03, 1.45772600e-03,
                   2.55102036e-03, 4.25170083e-03, 4.37317789e-03,
                   7.65306130e-03, 7.89601542e-03, 1.12973759e-02,
                   1.48202134e-02, 2.07725950e-02, 2.55102050e-02,
                   3.29203121e-02, 3.89941707e-02, 4.93197292e-02,
                   5.43002933e-02, 6.58406243e-02, 7.59232268e-02,
                   8.39407220e-02, 9.62099135e-02, 1.10058308e-01,
                   1.19533531e-01, 1.24878526e-01, 1.37147710e-01,
                   1.45651117e-01, 1.52575314e-01, 1.58041790e-01,
                   1.70432463e-01, 1.76141888e-01, 1.82458699e-01,
                   1.87074825e-01, 1.92541301e-01, 2.00072885e-01,
                   2.02016518e-01, 2.09426627e-01, 2.13313892e

In [11]:
dir_param = tclient.get_parameter("dir")

In [12]:
dir_param.value

0.0

In [13]:
dir_param.value = 70

In [ ]:
dir_param.value = 30
dp.get_slice("<comp(c)>", "chempotA")

Connection closed.
Attempt connection. localhost:34450
Connected, sending handshake.
Got HANDSHAKE_ACK. Server version 0 revision 0
ParameterSpace already registered: 'tinc::IdObject@1854295611592'
ParameterSpace already registered: 'tinc::IdObject@1854295611592'
ParameterSpace already registered: 'tinc::IdObject@1854295611592'
ParameterSpace already registered: 'tinc::IdObject@1854295611592'
ParameterSpace already registered: 'tinc::IdObject@1854295611592'


In [6]:
neighborhood = tclient.get_datapool("neighborhood")

In [7]:
neighborhood

In [9]:
neighborhood.get_current_files()

['C:\\Users\\Andres\\source\\repos\\vdv_data\\new\\kinetic_mc\\102\\conditions.0\\../cleared_path0.nc', 'C:\\Users\\Andres\\source\\repos\\vdv_data\\new\\kinetic_mc\\102\\conditions.0\\../free_range_path0.nc']

In [6]:
dp.slice_cache_dir

NameError: name 'dp' is not defined

In [13]:
param=tclient.parameter_spaces[0]
root_path=param.get_root_path()
trajectories_path=root_path+param.get_current_path()+"trajectory.nc"

KeyboardInterrupt: 

In [ ]:
trajectories_path

In [10]:
import netCDF4
nc = netCDF4.Dataset(trajectories_path)
nc.variables['occupation_dofs']
#choose an atom to track
#find the first step that site index changes, record new position
#track that position from then on
# repeat if changes.
atom_to_track=0
steps_changed=[]
previous_dof=nc.variables['occupation_dofs'][2000].data
print(previous_dof)

[0 1 0 ... 0 0 0]


In [11]:
len(nc.variables['occupation_dofs'][2000].data)

24696

In [22]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
!ncdump -c C:\\Users\\Andres\\source\\repos\\vdv_data\\MgTiS_spinel_diffusion_visualization\\event_tracker_testing_quick\\24\\conditions.0\\trajectory.nc

netcdf trajectory {
dimensions:
	sample_dim = 3000 ;
	config_dim = 24696 ;
variables:
	int passes(sample_dim) ;
	int steps(sample_dim) ;
	ubyte occupation_dofs(sample_dim, config_dim) ;
data:
}


In [13]:
time = zeros((3000, 24696))

In [3]:
import xarray as xr

In [12]:
ds = xr.open_dataset(trajectories_path, chunks = {'sample_dim':10})

In [16]:
equals = []
for i in range(len(ds['occupation_dofs']) -1):
    equals.append(ds['occupation_dofs'][i] == ds['occupation_dofs'][i + 1])

In [24]:
%%timeit
argwhere(equals)

The slowest run took 383.14 times longer than the fastest. This could mean that an intermediate result is being cached.
12h 47min 13s ± 19h 29min 29s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
list_of_occ_dofs=[]
for time_slice in nc.variables['occupation_dofs']:
    print (".")
    list_of_occ_dofs.append(time_slice)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


KeyboardInterrupt: 

In [ ]:
for step_count in range(nc.variables['occupation_dofs'].shape[0]):
    new_dof=nc.variables['occupation_dofs'][step_count].data
    if previous_dof[atom_to_track]!=new_dof[atom_to_track]:
        steps_changed.append(step_count)
        for i in len(previous_dof):
            if i!=atom_to_track and previous_dof[i]!=new_dof[i]:
                atom_to_track=i
               
    previous_dof=new_dof
   
print(steps_changed)